modelo.
voy a usar 2 datasets, los datos limpios que tengo exportados
y el csv de keywords

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [7]:
# importo set limpio 
columnas = ['title', 'genres', 'id', 'release_year', 'runtime', 'popularity', 'cast', 'crew']
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv", usecols = columnas)

In [5]:
df_movies.shape

(45375, 6)

In [8]:
columnas = ['popularity', 'release_year']  # Nombres de las columnas por las cuales deseas ordenar
df_movies = df_movies.sort_values(by=columnas, ascending=False)

In [384]:
df_movies.shape

(45375, 6)

In [9]:
df_movies.head(2)

,genres,id,popularity,runtime,title,cast,crew,release_year
30673,"['Family', 'Animation', 'Adventure', 'Comedy']",211672,547.488298,91.0,Minions,"['Sandra Bullock', 'Jon Hamm', 'Michael Keaton...","['Kyle Balda', 'Pierre Coffin']",2015
33325,"['Action', 'Adventure', 'Fantasy']",297762,294.337037,141.0,Wonder Woman,"['Gal Gadot', 'Chris Pine', 'Robin Wright', 'D...",['Patty Jenkins'],2017


In [10]:
# elimino cortos y series
df_movies = df_movies[(df_movies['runtime'] == 0.0) | (df_movies['runtime']>40)]

In [ ]:
df_movies.head(50)

In [386]:
df_movies.shape

(43616, 6)

In [11]:
df_movies['genres'] = df_movies['genres'].str.replace("[\[\]',]", "", regex=True)
df_movies['cast'] = df_movies['cast'].str.replace("[\[\]',]", "", regex=True)
df_movies['crew'] = df_movies['crew'].str.replace("[\[\]',]", "", regex=True)



REDUZCO LA CANTIDAD DE REGISTROS




In [12]:
df_movies = df_movies.head(6000)

In [13]:
#agrego indice
df_movies.reset_index(drop=True, inplace=True)
df_movies.reset_index(inplace=True)

In [14]:
df_movies

,index,genres,id,popularity,runtime,title,cast,crew,release_year
0,0,Family Animation Adventure Comedy,211672,547.488298,91.0,Minions,Sandra Bullock Jon Hamm Michael Keaton Allison...,Kyle Balda Pierre Coffin,2015
1,1,Action Adventure Fantasy,297762,294.337037,141.0,Wonder Woman,Gal Gadot Chris Pine Robin Wright Danny Huston...,Patty Jenkins,2017
2,2,Family Fantasy Romance,321612,287.253654,129.0,Beauty and the Beast,Emma Watson Dan Stevens Luke Evans Kevin Kline...,Bill Condon,2017
3,3,Action Crime,339403,228.032744,113.0,Baby Driver,Ansel Elgort Lily James Kevin Spacey Jamie Fox...,Edgar Wright,2017
4,4,Adventure Family Animation Action Comedy,177572,213.849907,102.0,Big Hero 6,Scott Adsit Ryan Potter Daniel Henney T.J. Mil...,Chris Williams Don Hall,2014
...,...,...,...,...,...,...,...,...,...
5995,5995,TV Movie Family Drama,57210,7.073059,87.0,Angels Sing,Chandler Canterbury Willie Nelson Harry Connic...,Tim McCanlies,2013
5996,5996,Adventure Fantasy Animation Science Fiction Fa...,12600,7.072301,75.0,Pokémon 4Ever: Celebi - Voice of the Forest,Veronica Taylor Rachael Lillis Maddie Blaustei...,Kunihiko Yuyama Jim Malone,2001
5997,5997,Thriller,77495,7.071300,107.0,Sleep Tight,Luis Tosar Marta Etura Alberto San Juan Petra...,Jaume Balagueró,2011
5998,5998,Western,6644,7.070669,120.0,El Dorado,John Wayne Robert Mitchum James Caan Charlene ...,Howard Hawks,1966




CONSTRUCCION DEL MODELO




In [15]:
# selecting the relevant features for recommendation

selected_features = ['cast','title', 'crew', 'genres']
print(selected_features)

['cast', 'title', 'crew', 'genres']


In [16]:
# replacing the null valuess with null string

for feature in selected_features:
  df_movies[feature] = df_movies[feature].fillna('')

In [17]:
combined_features = df_movies['cast']+ ' ' + df_movies['title']+ ' ' + df_movies['genres']+ ' ' + df_movies['crew']

In [18]:
print(combined_features)

0       Sandra Bullock Jon Hamm Michael Keaton Allison...
1       Gal Gadot Chris Pine Robin Wright Danny Huston...
2       Emma Watson Dan Stevens Luke Evans Kevin Kline...
3       Ansel Elgort Lily James Kevin Spacey Jamie Fox...
4       Scott Adsit Ryan Potter Daniel Henney T.J. Mil...
                              ...                        
5995    Chandler Canterbury Willie Nelson Harry Connic...
5996    Veronica Taylor Rachael Lillis Maddie Blaustei...
5997    Luis Tosar Marta Etura  Alberto San Juan Petra...
5998    John Wayne Robert Mitchum James Caan Charlene ...
5999    Rinal Mukhametov Sergey Garmash Oleg Menshikov...
Length: 6000, dtype: object


In [19]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

In [20]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [21]:
print(feature_vectors)

  (0, 9091)	0.049404944528317916
  (0, 496)	0.0641352241057248
  (0, 1639)	0.08563262002511461
  (0, 14045)	0.0761937647905618
  (0, 30002)	0.21130523889736236
  (0, 333)	0.15728353851310442
  (0, 2650)	0.13932464763533944
  (0, 3112)	0.3900861733298565
  (0, 24510)	0.22129073925759307
  (0, 43727)	0.1898078429779725
  (0, 33906)	0.06503769407484716
  (0, 12185)	0.20179248966036956
  (0, 966)	0.08999882634765063
  (0, 37490)	0.1713095875098209
  (0, 10598)	0.10806125517091648
  (0, 38384)	0.17354569074553847
  (0, 19449)	0.1422247950013977
  (0, 3759)	0.16739027710644058
  (0, 30189)	0.16926818519550146
  (0, 22641)	0.15300603296306742
  (0, 8939)	0.34709138149107693
  (0, 34575)	0.2219379585224131
  (0, 7254)	0.14856616464476033
  (0, 37860)	0.1339805344890819
  (0, 16253)	0.1144771920760538
  :	:
  (5999, 37747)	0.1710177660745865
  (5999, 24340)	0.1710177660745865
  (5999, 41676)	0.1710177660745865
  (5999, 30921)	0.1710177660745865
  (5999, 36999)	0.1710177660745865
  (5999, 15927)

In [ ]:
df_movies.info()

In [22]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [23]:
print(similarity)

[[1.         0.0143928  0.02567233 ... 0.         0.00548657 0.        ]
 [0.0143928  1.         0.04392135 ... 0.         0.03138179 0.00290186]
 [0.02567233 0.04392135 1.         ... 0.         0.01036281 0.00359623]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.00548657 0.03138179 0.01036281 ... 0.         1.         0.        ]
 [0.         0.00290186 0.00359623 ... 0.         0.         1.        ]]


In [27]:
print(similarity.shape)

(15000, 15000)


#### busco similitud por nombre

In [24]:
#movie_name = "Titanic"
titulo_pelicula= "The Terminator"


In [25]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = df_movies['title'].tolist()
print(list_of_all_titles)

['Minions', 'Wonder Woman', 'Beauty and the Beast', 'Baby Driver', 'Big Hero 6', 'Deadpool', 'Guardians of the Galaxy Vol. 2', 'Avatar', 'John Wick', 'Gone Girl', 'The Hunger Games: Mockingjay - Part 1', 'War for the Planet of the Apes', 'Captain America: Civil War', 'Pulp Fiction', 'Pirates of the Caribbean: Dead Men Tell No Tales', 'The Dark Knight', 'Blade Runner', 'The Avengers', 'Captain Underpants: The First Epic Movie', 'The Circle', 'The Bad Batch', 'The Maze Runner', 'Dawn of the Planet of the Apes', 'Alien: Covenant', 'Ghost in the Shell', 'Boyka: Undisputed IV', 'Whiplash', 'Fight Club', 'What Happened to Monday', 'Wish Upon', 'Thor: Ragnarok', 'Logan', 'Guardians of the Galaxy', '47 Meters Down', 'The Shawshank Redemption', 'The Last King', 'The Dark Tower', 'John Wick: Chapter 2', 'The Fate of the Furious', 'Forrest Gump', 'Pirates of the Caribbean: The Curse of the Black Pearl', 'Security', 'The Legend of Tarzan', 'King Arthur: Legend of the Sword', 'Doctor Strange', 'Sui

In [26]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(titulo_pelicula, list_of_all_titles)
print(find_close_match)

['The Terminator', 'The Terminal', 'The Terror']


In [27]:
close_match = find_close_match[0]
print(close_match)

The Terminator


In [28]:
# finding the index of the movie with title
index_of_the_movie = df_movies[df_movies.title == close_match]['index'].values[0]
print(index_of_the_movie)

305


In [29]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.007642359488862622), (1, 0.01370744460909372), (2, 0.033985288451424005), (3, 0.024320567853499963), (4, 0.033429666296776725), (5, 0.0515295245966819), (6, 0.03043977235512305), (7, 0.034795412354246505), (8, 0.02144024111423723), (9, 0.018901197368046706), (10, 0.027819488462261652), (11, 0.02159584866865725), (12, 0.040682336681570716), (13, 0.038529701311095926), (14, 0.026747313730683796), (15, 0.04596595575549711), (16, 0.05032628908725759), (17, 0.02680157329496665), (18, 0.027269989466036385), (19, 0.050707870079590746), (20, 0.015255324043439896), (21, 0.028844956563992153), (22, 0.02549953601293186), (23, 0.028385304148154933), (24, 0.020709846969533707), (25, 0.004509393544810673), (26, 0.006647415912052854), (27, 0.033690146831534734), (28, 0.01716831196412205), (29, 0.00449668207665512), (30, 0.038472622919070304), (31, 0.029461898222468422), (32, 0.030085607265286668), (33, 0.0030798577132896974), (34, 0.0615319415693342), (35, 0.0049608524077741235), (36, 0.023839

In [424]:
len(similarity_score)

15000

In [30]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(305, 0.9999999999999999), (195, 0.1592076012494599), (5005, 0.14368240389908912), (233, 0.123535567043579), (2029, 0.11091732589392109), (212, 0.11081426421751703), (2618, 0.10297154131383024), (4566, 0.09713649007356916), (3135, 0.09537965021745697), (633, 0.0912572622732141), (3505, 0.09068426335669198), (4960, 0.09053123507654781), (670, 0.08900116205613301), (1652, 0.0882616097256229), (257, 0.086767198788347), (504, 0.08516461269242066), (2855, 0.08302463352275041), (1580, 0.08143571724179981), (2987, 0.07916010196220052), (3532, 0.07840239499463356), (551, 0.07662215266462699), (115, 0.07632366338687781), (2556, 0.07571069298817107), (4343, 0.07408714002795329), (3502, 0.07361129007009613), (1333, 0.0732842537539929), (2719, 0.0723714019819362), (108, 0.07233087548574407), (2523, 0.07219492391068373), (5839, 0.07210593665164555), (1780, 0.07057399362858938), (2785, 0.06971077144670831), (1255, 0.06795860347623185), (306, 0.06782785291041134), (522, 0.06772686160158403), (1031, 

In [31]:
# resultados

print('Peliculas sugeridas : \n')

i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df_movies[df_movies.index==index]['title'].values[0]
  if (i<6):
    print(i, '.',title_from_index)
    i+=1

Peliculas sugeridas : 

1 . The Terminator
2 . Terminator 2: Judgment Day
3 . Superman: Doomsday
4 . Terminator 3: Rise of the Machines
5 . True Lies


In [445]:
print('Movies suggested for you:\n')

top_movies = sorted_similar_movies[:5]  # Obtener los primeros 5 elementos
indices = [movie[0] for movie in top_movies]
df_movies[df_movies.index==index]['title'].values[0]
#title_from_index= df_movies.loc[indices, 'title'].values    esto anda tambien
df_movies['rank'] = range(1, 6)
for i, movie_title in enumerate(df_movies['title_from_index'].head(5), 1):
    print(i, '.', movie_title)



Movies suggested for you:



ValueError: Length of values (5) does not match length of index (15000)

In [439]:
print('Movies suggested for you:\n')

top_movies = sorted_similar_movies[:5]  # Obtener los primeros 5 elementos

# Obtener los índices de las películas
indices = [movie[0] for movie in top_movies]

# Agregar la columna 'title_from_index' con los títulos correspondientes
df_movies['title_from_index'] = df_movies.loc[indices, 'title'].values

# Agregar la columna 'rank' para el ranking de las películas
df_movies['rank'] = range(1, 6)

# Imprimir las películas sugeridas
for i, movie_title in enumerate(df_movies['title_from_index'].head(5), 1):
    print(i, '.', movie_title)


Movies suggested for you:



ValueError: Length of values (5) does not match length of index (15000)

In [437]:
print('Movies suggested for you : \n')

top_movies = sorted_similar_movies[:5]  # Obtener los primeros 5 elementos

df_movies['title_from_index'] = df_movies.loc[top_movies[:, 0], 'title'].values  # Agregar la columna 'title_from_index' con los títulos correspondientes

df_movies['rank'] = range(1, 6)  # Agregar la columna 'rank' para el ranking de las películas

# Imprimir las películas sugeridas
df_movies[['rank', 'title_from_index']].apply(lambda x: print(x['rank'], '.', x['title_from_index']), axis=1)


Movies suggested for you : 



TypeError: list indices must be integers or slices, not tuple

In [438]:
print(top_movies)

[(317, 1.0000000000000002), (448, 0.8340750730379849), (707, 0.5298352523122352), (2871, 0.5265468207200843), (15, 0.467624539202325)]
